<a href="https://colab.research.google.com/github/LatifulJoy/covid19-cnn-TL-mobilenet/blob/main/Ensemble_Learning_For_Chest_Xray_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import pathlib 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import random
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from google.colab import files
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"latifuljoy","key":"c9493e600d8dfc76ec7a58ea29031a33"}'}

In [3]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

In [4]:
!kaggle datasets download -d latifuljoy/covid-chest-xray-image-dataset

 99% 480M/484M [00:07<00:00, 96.4MB/s]
100% 484M/484M [00:07<00:00, 66.6MB/s]


In [5]:
!unzip \*.zip

Archive:  covid-chest-xray-image-dataset.zip
  inflating: COVID_IEEE/covid/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg  
  inflating: COVID_IEEE/covid/03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg  
  inflating: COVID_IEEE/covid/1-s2.0-S0140673620303706-fx1_lrg.jpg  
  inflating: COVID_IEEE/covid/1-s2.0-S0929664620300449-gr2_lrg-a.jpg  
  inflating: COVID_IEEE/covid/1-s2.0-S0929664620300449-gr2_lrg-b.jpg  
  inflating: COVID_IEEE/covid/1-s2.0-S0929664620300449-gr2_lrg-c.jpg  
  inflating: COVID_IEEE/covid/1-s2.0-S0929664620300449-gr2_lrg-d.jpg  
  inflating: COVID_IEEE/covid/1-s2.0-S1684118220300608-main.pdf-001.jpg  
  inflating: COVID_IEEE/covid/1-s2.0-S1684118220300608-main.pdf-002.jpg  
  inflating: COVID_IEEE/covid/1-s2.0-S1684118220300682-main.pdf-002-a1.png  
  inflating: COVID_IEEE/covid/1-s2.0-S1684118220300682-main.pdf-002-a2.png  
  inflating: COVID_IEEE/covid/1-s2.0-S1684118220300682-main.pdf-003-b1.png  
  inflating: COVID_IEEE/covid/1-s2.0-S1684118220300682-main.pdf-003-b2.png 

In [6]:
for dirpath,dirnames,filenames in os.walk("/content/COVID_IEEE"):
    print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

there are 3 directories and 0 images in '/content/COVID_IEEE'.
there are 0 directories and 536 images in '/content/COVID_IEEE/virus'.
there are 0 directories and 536 images in '/content/COVID_IEEE/covid'.
there are 0 directories and 536 images in '/content/COVID_IEEE/normal'.


In [7]:
import pathlib
import tensorflow as tf
data_dir = pathlib.Path("/content/COVID_IEEE")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names

array(['covid', 'normal', 'virus'], dtype='<U6')

In [8]:
img_height =224
img_width =224

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = 'training',
    seed =123,
    image_size = (img_height, img_width),
    batch_size = 32
)

Found 1493 files belonging to 3 classes.
Using 1195 files for training.


In [10]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = 'validation',
    seed =123,
    image_size = (img_height, img_width),
    batch_size = 32
)

Found 1493 files belonging to 3 classes.
Using 298 files for validation.


In [11]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [12]:
import numpy as np
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x),y))
image_batch, label_batch= next(iter(normalized_ds))
first_image = image_batch[0]
#Notice the pixel values are now in (0,1)
print(np.min(first_image), np.max(first_image))

0.0 0.9996169


In [13]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds= train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
validation_ds= validation_ds.cache().prefetch(buffer_size = AUTOTUNE)

# Developed CNN Model

In [14]:
import tensorflow as tf
opt=tf.keras.optimizers.Adam(learning_rate=0.0001)

In [15]:
model = Sequential()

#Block Number 1
model.add(Conv2D(input_shape = (224,224,3), filters=32,padding="same", kernel_size= (3,3)))
model.add(Activation("relu"))

model.add(Conv2D(filters=32,padding="same", kernel_size= (3,3)))
model.add(Activation("relu"))

model.add(MaxPool2D((2,2)))

#Block Number 2
model.add(Conv2D(filters=64,padding="same", kernel_size= (3,3)))
model.add(Activation("relu"))


model.add(Conv2D(filters=64,padding="same", kernel_size= (3,3)))
model.add(Activation("relu"))

model.add(MaxPool2D((2,2)))
model.add(MaxPool2D((2,2)))


#Block Number 3
model.add(Conv2D(filters=128,padding="same", kernel_size= (3,3)))
model.add(Activation("relu"))

model.add(Conv2D(filters=128,padding="same", kernel_size= (3,3)))
model.add(Activation("relu"))

model.add(MaxPool2D((2,2)))

model.add(MaxPool2D((2,2)))


#Block Number 4
model.add(Conv2D(filters=256,padding="same", kernel_size= (3,3)))
model.add(Activation("relu"))

model.add(Conv2D(filters=256,padding="same", kernel_size= (3,3)))
model.add(Activation("relu"))

model.add(MaxPool2D((2,2)))

model.add(MaxPool2D((2,2)))

Dropout(0.40)


# Fully Connected layer
model.add(Flatten())

model.add(Dense(units=1024, activation="relu"))

model.add(Dense(units= 256, activation="relu"))
model.add(Dense(units=128, activation="relu"))
Dropout(0.40)

model.add(Dense(units=3, activation="softmax"))

In [21]:
#model.compile(optimizer= opt ,loss='kullback_leibler_divergence',metrics=['accuracy'])

In [16]:
model.compile(optimizer = opt, loss = tf.losses.SparseCategoricalCrossentropy(from_logits = False),metrics=['accuracy'])

In [17]:
history1 = model.fit(
    train_ds,
    validation_data = validation_ds,
    epochs=20
)

Epoch 1/20
38/38 [==============================] - 29s 209ms/step - loss: 0.7289 - accuracy: 0.6762 - val_loss: 0.4985 - val_accuracy: 0.8255
Epoch 2/20
38/38 [==============================] - 4s 115ms/step - loss: 0.4212 - accuracy: 0.8427 - val_loss: 0.4660 - val_accuracy: 0.8121
Epoch 3/20
38/38 [==============================] - 4s 118ms/step - loss: 0.2787 - accuracy: 0.9004 - val_loss: 0.2741 - val_accuracy: 0.9195
Epoch 4/20
38/38 [==============================] - 4s 116ms/step - loss: 0.1967 - accuracy: 0.9389 - val_loss: 0.2159 - val_accuracy: 0.9430
Epoch 5/20
38/38 [==============================] - 4s 117ms/step - loss: 0.1521 - accuracy: 0.9523 - val_loss: 0.2327 - val_accuracy: 0.9128
Epoch 6/20
38/38 [==============================] - 4s 117ms/step - loss: 0.1507 - accuracy: 0.9481 - val_loss: 0.2297 - val_accuracy: 0.9262
Epoch 7/20
38/38 [==============================] - 4s 118ms/step - loss: 0.0852 - accuracy: 0.9732 - val_loss: 0.2979 - val_accuracy: 0.8893
Epoch

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
model.save("/content/drive/MyDrive/model.hdf5")

# TF MODEL InceptionV3

In [20]:
import tensorflow as tf
opt=tf.keras.optimizers.Adam(learning_rate=0.0001)

In [21]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D,  GlobalAveragePooling2D
base_model = InceptionV3(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top = False)

#Freeze the first ten layers
for layer in base_model.layers[:10]:
  layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x= Dense(512, activation = 'relu')(x)
x= Dropout(0.4)(x)
predictions  = Dense(3, activation = 'softmax')(x)
model1 = Model(inputs = base_model.inputs, outputs = predictions)

87924736/87910968 [==============================] - 1s 0us/step


In [22]:
model1.compile(optimizer =opt, loss = tf.losses.SparseCategoricalCrossentropy(from_logits = False),metrics=['accuracy'])

In [23]:
history1 = model1.fit(
    train_ds,
    validation_data = validation_ds,
    epochs=15
)

Epoch 1/15
38/38 [==============================] - 23s 366ms/step - loss: 0.4820 - accuracy: 0.8042 - val_loss: 1.7563 - val_accuracy: 0.4094
Epoch 2/15
38/38 [==============================] - 9s 244ms/step - loss: 0.1230 - accuracy: 0.9607 - val_loss: 1.0369 - val_accuracy: 0.6208
Epoch 3/15
38/38 [==============================] - 9s 247ms/step - loss: 0.0352 - accuracy: 0.9900 - val_loss: 0.5367 - val_accuracy: 0.8289
Epoch 4/15
38/38 [==============================] - 9s 241ms/step - loss: 0.0110 - accuracy: 0.9967 - val_loss: 0.3405 - val_accuracy: 0.9295
Epoch 5/15
38/38 [==============================] - 9s 240ms/step - loss: 0.0066 - accuracy: 0.9983 - val_loss: 0.3136 - val_accuracy: 0.9396
Epoch 6/15
38/38 [==============================] - 9s 245ms/step - loss: 0.0064 - accuracy: 0.9992 - val_loss: 0.2460 - val_accuracy: 0.9497
Epoch 7/15
38/38 [==============================] - 9s 242ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.2384 - val_accuracy: 0.9530
Epoch

In [24]:
model1.save("/content/drive/MyDrive/model1.hdf5")

# DenseNet121

In [25]:
import tensorflow as tf
opt=tf.keras.optimizers.Adam(learning_rate=0.0001)

In [26]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D,  GlobalAveragePooling2D
base_model = DenseNet121(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top = False)

#Freeze the first ten layers
for layer in base_model.layers[:10]:
  layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x= Dense(512, activation = 'relu')(x)
x= Dropout(0.4)(x)
predictions  = Dense(3, activation = 'softmax')(x)
model2 = Model(inputs = base_model.inputs, outputs = predictions)

29097984/29084464 [==============================] - 0s 0us/step


In [27]:
model2.compile(optimizer =opt, loss = tf.losses.SparseCategoricalCrossentropy(from_logits = False),metrics=['accuracy'])

In [28]:
history2 = model2.fit(
    train_ds,
    validation_data = validation_ds,
    epochs=15
)

Epoch 1/15
38/38 [==============================] - 34s 527ms/step - loss: 0.3714 - accuracy: 0.8603 - val_loss: 0.4100 - val_accuracy: 0.8255
Epoch 2/15
38/38 [==============================] - 14s 367ms/step - loss: 0.0562 - accuracy: 0.9849 - val_loss: 0.2811 - val_accuracy: 0.8893
Epoch 3/15
38/38 [==============================] - 14s 374ms/step - loss: 0.0166 - accuracy: 0.9975 - val_loss: 0.1981 - val_accuracy: 0.9295
Epoch 4/15
38/38 [==============================] - 15s 381ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.1699 - val_accuracy: 0.9530
Epoch 5/15
38/38 [==============================] - 14s 371ms/step - loss: 0.0057 - accuracy: 0.9992 - val_loss: 0.1402 - val_accuracy: 0.9463
Epoch 6/15
38/38 [==============================] - 14s 369ms/step - loss: 0.0074 - accuracy: 0.9983 - val_loss: 0.1388 - val_accuracy: 0.9497
Epoch 7/15
38/38 [==============================] - 14s 366ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1480 - val_accuracy: 0.9597

In [29]:
model2.save("/content/drive/MyDrive/model2.hdf5")

# #TF model VGG16

In [ ]:
##from tensorflow.keras.applications import VGG16
#from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D,  GlobalAveragePooling2D
base_model = VGG16(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top = False)

#Freeze the first ten layers
for layer in base_model.layers[:10]:
  layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x= Dense(512, activation = 'relu')(x)
x= Dropout(0.3)(x)
#predictions  = Dense(8, activation = 'softmax')(x)
#model2 = Model(inputs = base_model.inputs, outputs = predictions)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
#checkpoint to save best model per epoch

model_filepath = "/content/drive/MyDrive-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint =ModelCheckpoint(
    filepath = model_filepath,
    monitor = 'val accuracy',
    mode = 'max',
    save_best_only = True,
    verbos =1
)

# models fit

In [ ]:
history1 = model1.fit(
    train_ds,
    validation_data = validation_ds,
    epochs=5, callbacks =[checkpoint] 
)

Epoch 1/5
43/43 [==============================] - ETA: 0s - loss: 0.5041 - accuracy: 0.8252

43/43 [==============================] - 18s 268ms/step - loss: 0.5041 - accuracy: 0.8252 - val_loss: 45.1035 - val_accuracy: 0.2903
Epoch 2/5
43/43 [==============================] - ETA: 0s - loss: 0.2639 - accuracy: 0.9232

43/43 [==============================] - 10s 242ms/step - loss: 0.2639 - accuracy: 0.9232 - val_loss: 6.0537 - val_accuracy: 0.4340
Epoch 3/5
43/43 [==============================] - ETA: 0s - loss: 0.1713 - accuracy: 0.9495

43/43 [==============================] - 11s 244ms/step - loss: 0.1713 - accuracy: 0.9495 - val_loss: 2.7602 - val_accuracy: 0.7977
Epoch 4/5
43/43 [==============================] - ETA: 0s - loss: 0.1052 - accuracy: 0.9642

43/43 [==============================] - 11s 242ms/step - loss: 0.1052 - accuracy: 0.9642 - val_loss: 4.9766 - val_accuracy: 0.7185
Epoch 5/5
43/43 [==============================] - ETA: 0s - loss: 0.1147 - accuracy: 0.9612

43/43 [==============================] - 10s 241ms/step - loss: 0.1147 - accuracy: 0.9612 - val_loss: 0.5829 - val_accuracy: 0.8446


In [ ]:
model1.save("/content/drive/MyDrive/model1.hdf5")

In [ ]:
history2 = model2.fit(
    train_ds,
    validation_data = validation_ds,
    epochs=5, callbacks =[checkpoint] 
)

Epoch 1/5
43/43 [==============================] - ETA: 0s - loss: 12.3671 - accuracy: 0.3555

43/43 [==============================] - 21s 399ms/step - loss: 12.3671 - accuracy: 0.3555 - val_loss: 1.0590 - val_accuracy: 0.4076
Epoch 2/5
43/43 [==============================] - ETA: 0s - loss: 1.1303 - accuracy: 0.3775

43/43 [==============================] - 12s 281ms/step - loss: 1.1303 - accuracy: 0.3775 - val_loss: 1.0377 - val_accuracy: 0.4076
Epoch 3/5
43/43 [==============================] - ETA: 0s - loss: 1.0690 - accuracy: 0.4338

43/43 [==============================] - 12s 275ms/step - loss: 1.0690 - accuracy: 0.4338 - val_loss: 0.9447 - val_accuracy: 0.6217
Epoch 4/5
43/43 [==============================] - ETA: 0s - loss: 1.0461 - accuracy: 0.4850

43/43 [==============================] - 12s 270ms/step - loss: 1.0461 - accuracy: 0.4850 - val_loss: 0.7556 - val_accuracy: 0.7331
Epoch 5/5
43/43 [==============================] - ETA: 0s - loss: 4.1352 - accuracy: 0.4031

43/43 [==============================] - 11s 265ms/step - loss: 4.1352 - accuracy: 0.4031 - val_loss: 1.0249 - val_accuracy: 0.4106


In [ ]:
model2.save("/content/drive/MyDrive/model2.hdf5")

# Ensemble Model

In [31]:
import tensorflow as tf
opt=tf.keras.optimizers.Adam(learning_rate=0.0001)

In [30]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
model_1 = load_model("/content/drive/MyDrive/model.hdf5")
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='name_of_model_1')
model_2 = load_model('/content/drive/MyDrive/model1.hdf5')
model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name='name_of_model_2')

model_3 = load_model('/content/drive/MyDrive/model2.hdf5')
model_3 = Model(inputs=model_3.inputs,
                outputs=model_3.outputs,
                name='name_of_model_3')

models = [model_1, model_2, model_3]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

In [32]:
ensemble_model.compile(optimizer = opt, 
                       loss= tf.losses.SparseCategoricalCrossentropy(from_logits=False),
                       metrics = ['accuracy'])

In [33]:
history = ensemble_model.fit(
    train_ds,
    validation_data = validation_ds,
    epochs =15
)

Epoch 1/15
38/38 [==============================] - 52s 891ms/step - loss: 0.0106 - accuracy: 1.0000 - val_loss: 0.1134 - val_accuracy: 0.9597
Epoch 2/15
38/38 [==============================] - 27s 706ms/step - loss: 0.0230 - accuracy: 0.9950 - val_loss: 0.4283 - val_accuracy: 0.9329
Epoch 3/15
38/38 [==============================] - 28s 725ms/step - loss: 0.0481 - accuracy: 0.9908 - val_loss: 0.1298 - val_accuracy: 0.9631
Epoch 4/15
38/38 [==============================] - 28s 741ms/step - loss: 0.0368 - accuracy: 0.9950 - val_loss: 0.1325 - val_accuracy: 0.9698
Epoch 5/15
38/38 [==============================] - 27s 706ms/step - loss: 0.0206 - accuracy: 0.9967 - val_loss: 0.1373 - val_accuracy: 0.9732
Epoch 6/15
38/38 [==============================] - 27s 719ms/step - loss: 0.0132 - accuracy: 0.9958 - val_loss: 0.1377 - val_accuracy: 0.9564
Epoch 7/15
38/38 [==============================] - 27s 718ms/step - loss: 0.0127 - accuracy: 0.9992 - val_loss: 0.1324 - val_accuracy: 0.9698